In [1]:
import matplotlib.pyplot as plt
import networkx as nx
import textwrap
import numpy as np  # For calculating text rotation


# Define the etymology data for "extraordinary"
etymology_data = {
  "word": "exhausted",
  "meaning": "drained of one's physical or mental resources; very tired",
  "example_words": ["I'm exhausted after the workout.", "The resources are nearly exhausted."],
  "language": "English",
  "synonyms_in_other_languages": {
    "japanese": "疲れ果てた",
    "arabic": "مُنهَك",
    "french": "épuisé",
    "chinese": "筋疲力尽"
  },
  "tracing": [
    "exhausted <-- exhaust <-- ex- + haurire"
  ],
  "history": "Originally from Latin 'exhaurire' meaning 'to draw out' or 'to drain', it evolved into 'exhaust' in English, signifying the act of draining something completely.",
  "parts": [
    {
      "part": "ex-",
      "type": "prefix"
    },
    {
      "part": "haurire",
      "type": "root"
    }
  ],
  "etymology": [
    {
      "part": "ex-",
      "meaning": "out, away from",
      "example_words": ["exclude", "expel", "extend"],
      "language": "Latin",
      "history": "This prefix comes from Latin, indicating removal or separation.",
      "tracing": [
        "ex- <-- ex (Latin)"
      ],
      "etymology": [
        {
          "part": "ex",
          "meaning": "out of, from within",
          "language": "Latin",
          "history": "Rooted in Indo-European languages, indicating outward motion.",
          "example_words": ["exit", "exile"],
          "tracing": [
            "ex (Latin) <-- eghs (Proto-Indo-European)"
          ],
          "etymology": [
            {
              "part": "eghs",
              "meaning": "out",
              "language": "Proto-Indo-European",
              "history": "Ancient root denoting outward movement or direction.",
              "example_words": ["exodus (through Latin)"],
              "tracing": []
            }
          ]
        }
      ]
    },
    {
      "part": "haurire",
      "meaning": "to draw water, to draw out",
      "example_words": ["exhaust", "inexhaustible"],
      "language": "Latin",
      "history": "Used in Latin to describe the act of drawing out, especially liquids.",
      "tracing": [
        "haurire <-- haustus (Latin)"
      ],
      "etymology": [
        {
          "part": "haustus",
          "meaning": "a drawing out, draining",
          "language": "Latin",
          "history": "Past participle of haurire, used in various forms to indicate the act of drawing out or consuming.",
          "example_words": ["haustorium", "exhaustive"],
          "tracing": [
            "haustus <-- haurire (Latin)"
          ],
          "etymology": [
            {
              "part": "haurire",
              "meaning": "to draw, to drink",
              "language": "Latin",
              "history": "Reflects the physical act of drawing liquids, metaphorically extended to resources or energy.",
              "example_words": ["exhaust"],
              "tracing": [],
              "etymology": []
            }
          ]
        }
      ]
    }
  ]
}


# Initialize the directed graph and populate it
G = nx.DiGraph()

# def add_to_graph(graph, data, parent=None):
    
    
#     node_data = {key: data.get(key, '') for key in ['part', 'language', 'meaning', 'example_words']}
#     node = node_data['part']
#     graph.add_node(node, **node_data)
#     if parent:
#         graph.add_edge(parent, node)
#     for child in data.get('etymology', []):
#         add_to_graph(graph, child, parent=node)

# Update extraordinary_data with synonyms as example_words for the word part
etymology_data["example_words"] = list(etymology_data["synonyms_in_other_languages"].values())


# add_to_graph(G, {"part": etymology_data["word"], "language": etymology_data["language"], "meaning": etymology_data["meaning"], "etymology": etymology_data["etymology"]})


def add_to_graph_iterative(graph, data):
    queue = [(data, None)]  # Start with the root node data and no parent
    while queue:
        current_data, parent = queue.pop(0)  # FIFO queue
        node_data = {key: current_data.get(key, '') for key in ['part', 'language', 'meaning', 'example_words']}
        node = node_data['part']
        graph.add_node(node, **node_data)
        if parent:
            graph.add_edge(parent, node)
        for child in current_data.get('etymology', []):
            queue.append((child, node))  # Add child data and current node as parent

# Apply the updated function
add_to_graph_iterative(G, {"part": etymology_data["word"], "language": etymology_data["language"], "meaning": etymology_data["meaning"], "etymology": etymology_data["etymology"]})


# Initial positions: place the main word at an arbitrary top center, e.g., (0, 0)
pos = {etymology_data["word"]: (0, 0)}



# # Function to manually set positions for child nodes to avoid overlap
# def set_positions_branch_wise(graph, current_pos, parent, level=1, branch_offset=0):
#     children = list(graph.successors(parent))
#     num_children = len(children)
#     width_between_children = 1.0 / (num_children + 1)
#     for i, child in enumerate(children):
#         # Horizontal position: disperse children horizontally
#         x = -0.5 + width_between_children * (i + 1) + branch_offset
#         # Vertical position: place lower levels further down
#         y = -level * 0.1
#         current_pos[child] = (x, y)
#         # Recursively position the children of the current node, adjusting the branch offset
#         set_positions_branch_wise(graph, current_pos, child, level + 1, branch_offset + i * 0.1)

# # Apply custom positioning
# set_positions_branch_wise(G, pos, etymology_data["word"])

def set_positions_branch_wise_iterative(graph, initial_pos):
    # Use a list as a stack for DFS traversal, starting with the root node, level, and initial branch offset
    stack = [(etymology_data["word"], 0, 0, 0)]  # Each element is (node, level, branch_offset, index_in_parent)
    current_pos = initial_pos

    while stack:
        # Pop the last element (DFS)
        node, level, branch_offset, index_in_parent = stack.pop()
        num_children = len(list(graph.successors(node)))
        
        if num_children > 0:
            width_between_children = 1.0 / (num_children + 1)
        else:
            width_between_children = 0
        
        # Calculate this node's position
        x = -0.5 + width_between_children * (index_in_parent + 1) + branch_offset
        y = -level * 0.1
        current_pos[node] = (x, y)
        
        # Push children to the stack, with updated level and branch_offset
        for i, child in enumerate(graph.successors(node)):
            # Calculate new branch offset based on current node's position and child index
            new_branch_offset = branch_offset + i * width_between_children if num_children > 1 else branch_offset
            stack.append((child, level + 1, new_branch_offset, i))

    return current_pos

# Initialize positions with the root node
initial_pos = {etymology_data["word"]: (0, 0)}
# Apply the updated function
pos = set_positions_branch_wise_iterative(G, initial_pos)


# Adjust all positions so that the main word is at the top center of the final layout
# Find min and max x to center horizontally
min_x = min(pos.values(), key=lambda x: x[0])[0]
max_x = max(pos.values(), key=lambda x: x[0])[0]
for node in pos:
    # Centering horizontally
    pos[node] = ((pos[node][0] - min_x) / (max_x - min_x), pos[node][1])

plt.figure(figsize=(15, 10))

# Draw the graph
nx.draw_networkx_nodes(G, pos, node_size=3000, node_color="lightblue", alpha=0.7)
nx.draw_networkx_edges(G, pos, arrowstyle="<|-", arrowsize=30, edge_color="gray")
nx.draw_networkx_labels(G, pos, font_size=20, font_weight="bold")

# Add edge labels (keeping your original method)
for edge in G.edges:
    source, target = edge
    mid_x, mid_y = np.mean([pos[source], pos[target]], axis=0)
    angle = np.degrees(np.arctan2(pos[target][1] - pos[source][1], pos[target][0] - pos[source][0])) + 180
    plt.text(mid_x, mid_y, f"{G.nodes[target]['language']} --> {G.nodes[source]['language']}", rotation=angle, rotation_mode='anchor', fontsize=10, color='red', ha='center', va='center')

# Add example words around nodes
for node, (x, y) in pos.items():
    examples = ", ".join(G.nodes[node].get('example_words', []))
    wrapped_text = textwrap.fill(examples, width=20)
    plt.text(x, y - 0.15, wrapped_text, fontsize=14, ha='center', va='top', wrap=True)

plt.title(f'Etymology of "{etymology_data["word"]}"', fontsize=15)
plt.axis('off')
plt.show()


KeyboardInterrupt

